In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

from config import Config

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
import faiss
import numpy as np

In [ ]:
cnfg = Config()

In [ ]:
external_df = pd.read_csv(f"{cnfg.base_data_path}aimo_external/external_df.csv")

In [ ]:
external_df.head()

,problem,level,type,solution,stage,source
0,Kevin Kangaroo begins hopping on a number line...,Level 5,Algebra,Kevin hops $1/3$ of the remaining distance wit...,train,MATH
1,The ratio of the areas of two squares is $\fra...,Level 4,Algebra,We start off by simplifying the ratio $\frac{1...,train,MATH
2,"If $\sqrt{2\sqrt{t-2}} = \sqrt[4]{7 - t}$, the...",Level 4,Algebra,"We raise both sides to the fourth power, which...",train,MATH
3,Let $t(x) = \sqrt{3x+1}$ and $f(x)=5-t(x)$. Wh...,Level 4,Algebra,We first evaluate $f(5) = 5 -t(5) = 5-\sqrt{5\...,train,MATH
4,James has a total of 66 dollars in his piggy b...,Level 2,Algebra,Call the number of one dollar bills $x$ and th...,train,MATH


In [ ]:
model = SentenceTransformer("AnReu/albert-for-arqmath-3")
# model = AutoModelForSequenceClassification.from_pretrained("AnReu/albert-for-arqmath-3")
# tokenizer = AutoTokenizer.from_pretrained("AnReu/albert-for-arqmath-3")

No sentence-transformers model found with name AnReu/albert-for-arqmath-3. Creating a new one with mean pooling.
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at AnReu/albert-for-arqmath-3 were not used when initializing AlbertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificatio

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: AlbertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# inp = "A certain has $4$ digits when written in base $8$. The same integer has $d$ digits when written in base $2$. What is the sum of all possible values of $d$?"
# inputs = tokenizer.encode_plus(
#         inp,  # Sentence to encode.
#         add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
#         max_length=context_length,  # Pad & truncate all sentences.
#         truncation=True,
#         padding="max_length",
#         return_attention_mask=True,  # Construct attn. masks.
#         return_tensors="pt",
#     )
# outputs = model['encoder'](input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])

In [ ]:
# train_data_vec = []
# for i,row in tqdm(external_df.iterrows()):
#     embeddings = model.encode([row['problem']])
#     train_data_vec.append(embeddings[0])

In [ ]:
problem_list = list(external_df["problem"])
solution_list = list(external_df["solution"])

In [ ]:
problem_list[10], solution_list[10]

('If $3+a=4-b$ and $4+b=7+a$, what is $3-a$?',
 'First we begin by solving the system of equations \\begin{align*}\n3+a&=4-b, \\\\\n4+b&=7+a.\n\\end{align*}Adding the two equations, we get $3+a+4+b=4-b+7+a$, which simplifies to $7+a+b=11+a-b$. Cancelling $a$ from both sides, we get $7+b=11-b$. Solving for $b$, we find that $b=2$. Plugging this into the first equation above, we obtain $3+a=4-2$. Hence $a=-1$ and $3-a=\\boxed{4}$.')

In [ ]:
embeddings = model.encode(problem_list)

In [ ]:
print(embeddings.shape)
print("Embedding length", embeddings.shape[1])

(21292, 768)
Embedding length 768


In [ ]:
np.savez_compressed("compressed_array_embeddings.npz", array_data=embeddings)

In [ ]:
import os

# Get the size of the file in bytes
file_size_bytes = os.path.getsize("compressed_array_embeddings.npz")

# Convert bytes to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)

print("File size:", file_size_mb, "MB")

File size: 58.08137035369873 MB


In [ ]:
loaded_embeddings = np.load("compressed_array_embeddings.npz")

# Access the array by the name you specified ('my_array' in this case)
loaded_embeddings = loaded_embeddings["array_data"]

loaded_embeddings.shape

(21292, 768)

In [ ]:
external_df[["problem", "solution"]].to_csv(
    "compressed_dataframe.csv.gz", compression="gzip", index=False
)

In [ ]:
embed_length = loaded_embeddings.shape[1]

index = faiss.IndexFlatL2(embed_length)

index.add(loaded_embeddings)

In [ ]:
index.is_trained

True

In [ ]:
# Check the total number of embeddings in the index
index.ntotal

21292

In [ ]:
query_text = "For how many positive integers $m$ does the equation \[\vert \vert x-1 \vert -2 \vert=\frac{m}{100}\] have $4$ distinct solutions?"
query = [query_text]


# Vectorize the query string
query_embedding = model.encode(query)

In [ ]:
# Set the number of outputs we want
top_k = 3

# Run the query
# index_vals refers to the chunk_list index values
scores, index_vals = index.search(query_embedding, top_k)

In [ ]:
print(index_vals)
print(scores)

[[ 5311  4807 12413]]
[[2.5581956 2.5946705 2.6856375]]


In [ ]:
pred_indexes = index_vals[0]

In [ ]:
tmp = external_df.iloc[pred_indexes]

In [ ]:
faiss.__version__

'1.8.0'

In [ ]:
tmp.iloc[0].problem

'What is the largest three-digit integer $n$ that satisfies $$55n\\equiv 165\\pmod{260}~?$$'

In [ ]:
tmp.iloc[1].solution

'We have that \\[a\\equiv (3^{2n}+4)^{-1}\\equiv (9^{n}+4)^{-1}\\equiv 4^{-1}\\equiv \\boxed{7}\\pmod{9}.\\]'

In [ ]:
# tmp.iloc[3].problem